In [8]:
import numpy as np
from PIL import Image

In [10]:
from mnist import mnist
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

len(test_images)

10000

In [4]:
class Conv3x3:
    def __init__(self, num_filters):
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, 3, 3) / 9
        
    def iterate_regions(self, image):
        h, w = image.shape
        for i in range (h - 2):
            for j in range(w - 2):
                sample = image[i : (i + 3), j : (j + 3)]
                yield sample, i, j
                
    def forward(self, image):
        h, w = image.shape
        output = np.zeros((h - 2, w - 2, self.num_filters))
        for sample, i, j in self.iterate_regions(image):
            output[i, j] = np.sum(sample * self.filters, axis=(1, 2))
        return output

In [5]:
class MaxPool2:
    def iterate_regions(self, image):
        h, w, f = image.shape
        for i in range ((h - 1) // 2):
            for j in range((w - 1) // 2):
                sample = image[2*i : (2*i + 2), 2*j : (2*j + 2), :]
                yield sample, i, j
                
    def forward(self, image):
        h, w, f = image.shape
        output = np.zeros((h // 2, w // 2, f))
        for sample, i, j in self.iterate_regions(image):
            output[i, j] = np.amax(sample, axis=(0, 1))
        return output

In [6]:
class Softmax:
    def __init__(self, n_inputs, nodes):
        self.n_inputs = n_inputs
        self.nodes = nodes
        self.weights = np.random.randn(n_inputs, nodes) / n_inputs
        self.biases = np.zeros(nodes)
        
    def forward(self, input):
        input = input.flatten()
        z = np.dot(input, self.weights) + self.biases
        a = np.exp(z)
        return a / np.sum(a, axis = 0)

In [7]:
conv = Conv3x3(8)
pool = MaxPool2()
softmax = Softmax(13*13*8, 10)

def forward(image, label):
    out = conv.forward(image)
    out = pool.forward(out)
    out = softmax.forward(out)
    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0
    return out, loss, acc

loss = 0
num_correct = 0
avg_loss = 0
for i, (im, label) in enumerate(zip(test_images, test_labels)):
    _, l, acc = forward(im, label)
    num_correct += acc
    loss += l
    
    
    if(i % 100 == 99):
        print("[Step %d]: Average CE = %.3f, Accuracy = %d%%" % (i + 1, loss / 100, num_correct))
        avg_loss += loss
        loss = 0
        num_correct = 0
        
    if(i == 1000):
        print("Average Loss = %.3f" % (avg_loss / 1000))
        break

[Step 100]: Average CE = 2.464, Accuracy = 7%
[Step 200]: Average CE = 2.501, Accuracy = 9%
[Step 300]: Average CE = 2.568, Accuracy = 9%
[Step 400]: Average CE = 2.533, Accuracy = 9%
[Step 500]: Average CE = 2.594, Accuracy = 13%
[Step 600]: Average CE = 2.601, Accuracy = 11%
[Step 700]: Average CE = 2.615, Accuracy = 9%
[Step 800]: Average CE = 2.492, Accuracy = 13%
[Step 900]: Average CE = 2.674, Accuracy = 12%
[Step 1000]: Average CE = 2.531, Accuracy = 11%
Average Loss = 2.557
